In [2]:
from bs4 import BeautifulSoup
import bs4
from urllib.request import urlopen, Request
from urllib.error import HTTPError
import mechanicalsoup
import time
import os
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image
from selenium.common.exceptions import NoSuchElementException
import io
import requests
from selenium.common.exceptions import ElementClickInterceptedException
import pandas as pd
import re
chrome_path = '/Users/marissakhoury/Desktop/webscrape/drivers/chromedriver'

# Skin Care Project


## Getting the Data

#### Columns of Interest

- Product Brand
- Product Name
- Price
- Size 
- Product Description
- Skin Type
- Ingredients
- Rating
- Number of Reviews

In [2]:
def init_driver(url=""):
    driver = webdriver.Chrome('./drivers/chromedriver')
    driver.get(url)
    return driver

In [3]:
# this was the code I tried to use to parse details for the corrected_urls; it wasnt working
def parse_details(s): 
    if 'Which skin type is it good for?' in s: 
        try:
            detail = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/div[5]/div[2]').text
            pattern = r"✔ \w+\n"
            skin_type = re.findall(pattern, detail)
        except Exception:
            skin_type = None
        try:
            start_description = 'What it is:'
            start_string = s.index(start_description)
            end_string = s.index('.')
            product_description = s[start_string + len(start_description):(end_string + 1)]
        except Exception:
            product_description = None
    elif 'Skin Type:' in s: 
        start_skin_type = 'Skin Type: '
        start_index = s.index(start_skin_type)
        end_skin_type = s.index('\nSkincare Concerns:')
        skin_type = s[start_index + len(start_skin_type): end_skin_type]
        try:
            start_description = 'What it is:'
            start_string = s.index(start_description)
            end_string = s.index('.')
            product_description = s[start_string + len(start_description):(end_string + 1)]
        except Exception:
            product_description = None
    return skin_type, product_description

# Sephora

In [14]:
def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver

def parse_sephora_products():
    driver = webdriver.Chrome(executable_path = chrome_path)
    url = 'https://www.sephora.com'
    driver.get(url)

    # initiate empty dataframe

    df = pd.DataFrame(columns=['Label', 'URL'])
    # step 1
    tickers = ['moisturizing-cream-oils-mists', 'cleanser', 'facial-treatments', 'face-mask',
               'eye-treatment-dark-circle-treatment', 'sunscreen-sun-protection', 'self-tanning-products',
               'lip-treatments', 'vegan-skin-care']

    for ticker in tickers:
        url = 'https://www.sephora.com/shop/' + ticker + '?pageSize=300'
        driver.get(url)

        try:
            x_button = driver.find_element(By.XPATH, '//*[@id="modal0Dialog"]/button')
            ## how to perform actions 
            # https://www.geeksforgeeks.org/action-chains-in-selenium-python/
            ActionChains(driver).click(x_button).perform()
            time.sleep(5)
        except NoSuchElementException:
            pass

        for _ in range(10):
            browser = scrollDown(driver, 10)
            time.sleep(5)

        element = driver.find_elements_by_class_name('css-ix8km1')
        print(element)

        subpageURL = []
        for a in element:
            subURL = a.get_attribute('href')
            subpageURL.append(subURL)

        # transform into a data frame
        dic = {'Label': ticker, 'URL': subpageURL}
        print(dic)
        df = df.append(pd.DataFrame(dic), ignore_index = True)

    # add columns
    df2 = pd.DataFrame(columns=['brand', 'name', 'price', 'rank', 'skin_type', 'ingredients'])
    df = pd.concat([df, df2], axis = 1)
    print(df)
    return df

def parse_details(s): 
    skin_type = None
    product_description = s
    return skin_type, product_description

def parse_ingredients(s):
    ingredients = []
    for line in s.split('\n'):
        if line.startswith('-'):
            # main ingredient (focused)
            ingredient = line[line.find('-')+1 : line.find(':')]
            ingredients.append(ingredient)
        else:
            if not line:
                continue
            ingredients.extend([ing.strip() for ing in line.split(',')])
    return ingredients

def test_page(url):
    driver = webdriver.Chrome(executable_path = chrome_path)
    
    output = [] ## store the data here as a row
    # go to page 
    driver.get(url)
    time.sleep(5)
    
    # parse things
    brand = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/div[1]/div[1]/h1/a/span').text
    prod_name = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/div[1]/div[1]/h1/span').text
    price = float(driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/div[1]/div[2]/div[1]/span').text.replace('$', ' ').strip())   
    try:
        size = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/span').text
    except:
        size = None
        
    if not size:
        try:
            size = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/div[1]/div[1]/div[1]').text
        except:
            size = None
    
    if size:
        size_pattern = r'([0-9\.]+)\s*[fl\.oz]+'
        size = re.search(size_pattern, size)
        if size is not None:
            size = float(size.group(1))
    
#     SIZE 1.69 oz/ 50 mL
#     SIZE: 1.7 fl. oz/ 50 mL
    
    scrollDown(driver, 10)
    time.sleep(2)
    
    # xpath that holds body for Details, How to Use, etc.
    xpath_info_details = '/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/div[4]/div[2]/div'
    xpath_info_ingred = '/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/div[4]/div[4]/div'
    # description = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/div[4]/div[2]/div').text
    description = driver.find_element_by_id('tabpanel0').text
    prod_description, skin_type = parse_details(description)
    # get ingredients
    xpath = '/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/div[4]/div[1]/button[3]' 
    btn = driver.find_element_by_id('tab2')
    btn.click()
    ingredients = driver.find_element_by_id('tabpanel2').text
    ingredients = parse_ingredients(ingredients)
    # rating
    scrollDown(driver, 10)
    time.sleep(2)
    scrollDown(driver, 10)
    time.sleep(2)
    try:
        rating_out_of_5 = driver.find_element_by_xpath('//*[@id="ratings-reviews"]/div[2]/div[2]/div[1]/div/div/div[2]').text
        rating = rating_out_of_5[: rating_out_of_5.find(' /')]
    except Exception: 
        rating = None
    if rating == None:
        try:
            rating_out_of_5 = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div/main/div/div[2]/div[1]/div/div[2]/div[2]/div[1]/div/div/div[2]').text
            rating = rating_out_of_5[: rating_out_of_5.find(' /')]
        except Exception: 
            rating = None
    # number of reviews
    reviews = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/main/div/div[2]/div[1]/div/div[2]/div[1]/span').text
    num_reviews = reviews[: reviews.find(' ')]
    
    output = [brand, prod_name, price, size, prod_description, skin_type, ingredients, rating, num_reviews]
    driver.close()
    return output, None

In [69]:
# Webscraping product names and urls from Sephora ticker pages
data = parse_sephora_products()
data.head

,prod_name,URL,brand,name,price,rank,skin_type,ingredients
0,Ultra Facial Cream,https://www.sephora.com/product/ultra-facial-c...,NaN,NaN,NaN,NaN,NaN,NaN
1,Truth Serum®,https://www.sephora.com/product/truth-serum-P4...,NaN,NaN,NaN,NaN,NaN,NaN
2,Lactic Acid 10% + HA,https://www.sephora.com/product/the-ordinary-d...,NaN,NaN,NaN,NaN,NaN,NaN
3,Granactive Retinoid* 2% Emulsion,https://www.sephora.com/product/the-ordinary-d...,NaN,NaN,NaN,NaN,NaN,NaN
4,Ultimune Power Infusing Serum Concentrate,https://www.sephora.com/product/ultimune-power...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
228,Glow Cycle Retin-ALT Power Serum,https://www.sephora.com/product/glow-cycle-ret...,NaN,NaN,NaN,NaN,NaN,NaN
229,Self-Tanning Water,https://www.sephora.com/product/self-tanning-w...,NaN,NaN,NaN,NaN,NaN,NaN
230,Overtime Mask,https://www.sephora.com/product/overtime-mask-...,NaN,NaN,NaN,NaN,NaN,NaN
231,Instant Detox Mask,https://www.sephora.com/product/instant-detox-...,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# populating the dataframe by visiting each page
def populate_df(df):
    for i, ind in enumerate(df.index):
        url = df.URL[ind]
        if df.ingredients[ind] is None:
            continue
        try:
            data, _ = test_page(url)
        except Exception as e:
            print(ind, e)
            data = [None] * 9
        # add to df
        # [brand, prod_name, price, size, prod_description, skin_type, ingredients, rating, num_reviews]
        df.brand[ind] = data[0]
        df.name[ind] = data[1]
        df.price[ind] = data[2]
        df.prod_size[ind] = data[3]
        df.description[ind] = data[4]
        df.skin_type[ind] = data[5]
        df.ingredients[ind] = data[6]
        df.rating[ind] = data[7]
        df.num_reviews[ind] = data[8]
    return df

In [33]:
populate_df(corrected_df)

/Users/tinhnguyen/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tinhnguyen/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tinhnguyen/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/tinhnguyen/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A val

6 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[2]/div/main/div/div[2]/div[1]/div/div[2]/div[1]/span"}
  (Session info: chrome=87.0.4280.88)

128 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[2]/div/main/div/div[2]/div[1]/div/div[2]/div[1]/span"}
  (Session info: chrome=87.0.4280.88)

134 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/div[1]/div[1]/h1/a/span"}
  (Session info: chrome=87.0.4280.88)

162 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/div[1]/div[1]/h1/a/span"}
  (Session info: chrome=87.0.4280.88)

165 Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[2]/div/main/div/div[1]/div/div[2]/div[1]/div[1]/h1/a/span"}
  (Session info:

,prod_name,URL,brand,name,price,skin_type,ingredients,prod_size,description,num_reviews,rating
0,Ultra Facial Cream,https://www.sephora.com/product/ultra-facial-c...,Kiehl's Since 1851,Ultra Facial Cream,32.0,What it is: A bestselling face moisturizer tha...,"[Glacial Glycoprotein, Squalene, Aqua/Water, G...",1.70,NaN,1136.0,4.3
1,Truth Serum®,https://www.sephora.com/product/truth-serum-P4...,OLEHENRIKSEN,Truth Serum®,50.0,Which skin type is it good for?\n✔ Normal\n✔ O...,"[True-C Complex, Collagen, Orange and Green Te...",1.00,NaN,5263.0,4.2
2,Lactic Acid 10% + HA,https://www.sephora.com/product/the-ordinary-d...,The Ordinary,Lactic Acid 10% + HA,6.8,Which skin type is it good for?\n✔ Normal\n✔ O...,"[Lactic Acid, Tasmanian Pepperberry, Water, La...",1.00,NaN,837.0,4.5
3,Granactive Retinoid* 2% Emulsion,https://www.sephora.com/product/the-ordinary-d...,The Ordinary,Granactive Retinoid* 2% Emulsion,9.8,Which skin type is it good for?\n✔ Normal\n✔ O...,"[Granactive Retinoid, Water, Glycerin, Ethyl L...",1.00,NaN,549.0,4.5
4,Ultimune Power Infusing Serum Concentrate,https://www.sephora.com/product/ultimune-power...,Shiseido,Ultimune Power Infusing Serum Concentrate,110.0,Which skin type is it good for?\n✔ Normal\n✔ O...,"[ImuGeneration Technology, Exclusive Ultimune ...",1.60,NaN,430.0,4.1
5,Blue Tansy BHA and Enzyme Pore Refining Mask,https://www.sephora.com/product/superberry-hyd...,Youth To The People,Superberry Hydrate + Glow Dream Mask with Vita...,48.0,What it is: An overnight mask that thoroughly ...,"[Vitamin C (THD Ascorbate), Squalane and Hyalu...",2.00,NaN,1348.0,4.4
6,Capture Youth Serum Collection,https://www.sephora.com/product/capture-youth-...,None,None,NaN,None,None,NaN,NaN,NaN,NaN
7,Jet Lag Mask,https://www.sephora.com/product/jet-lag-mask-P...,Summer Fridays,Jet Lag Mask,48.0,Which skin type is it good for?\n✔ Normal\n✔ O...,"[Chestnut Extract, Vitamin C, Niacinamide, Wat...",2.25,NaN,3321.0,4.0
8,Face Mask,https://www.sephora.com/product/gravitymud-fir...,GLAMGLOW,GRAVITYMUD™ Firming Treatment Mask,60.0,Which skin type is it good for?\n✔ Normal\n✔ O...,"[Water, Polyvinyl Alcohol, Alcohol Denat., But...",1.70,NaN,852.0,3.9
9,Pitera Facial Treatment Mask,https://www.sephora.com/product/facial-treatme...,SK-II,Pitera Facial Treatment Mask,139.0,Which skin type is it good for?\n✔ Normal\n✔ O...,"[Pitera, Water, Saccharomycopsis Ferment Filtr...",NaN,NaN,397.0,4.3


In [1]:
corrected_df.to_csv('CORRECTED.csv')

NameError: name 'corrected_df' is not defined

In [3]:
corrected_df = pd.read_csv('CORRECTED.csv')

## Correcting the Incorrect URLs from Sephora

Our method of parsing product urls (via tickers) ran into issues where the url attached to certain listed products were incorrect, or led to a 404. Below, we've consolidated all the products that have this issue. We found these products by looking for rows where product specific information (brand, price, product size) was not parsed. 

In [7]:
corrected_df[corrected_df.prod_size.isna()].prod_name.tail()

205               EDP Cheirosa ’62 Holiday Fragrance Set
217    Clear Improvement® Active Charcoal Mask to Cle...
218    Let It Grow - Hair Sweet Hair Vegan Gummies Du...
221    Alpha Beta® Ultra Gentle Daily Peel for Sensit...
229                                   Self-Tanning Water
Name: prod_name, dtype: object

In [7]:
names_with_incorrect_urls = ['Ultra Facial Cream',
 'Truth Serum®',
 'Lactic Acid 10% + HA',
 'Granactive Retinoid* 2% Emulsion',
 'Ultimune Power Infusing Serum Concentrate',
 'Blue Tansy BHA and Enzyme Pore Refining Mask',
 'Capture Youth Serum Collection',
 'Jet Lag Mask',
 'Face Mask',
 'Pitera Facial Treatment Mask',
 'Water Sleeping Mask',
 'Original Skin™ Retexturizing Mask with Rose Clay',
 'Mud Mask Purifying & Mattifying',
 'Clear Improvement™ Charcoal Honey Mask to Purify and Nourish',
 'Aqua Bomb Sleeping Mask',
 'Out of Trouble™ 10 Minute Mask to Rescue Problem Skin',
 'GRAVITYMUD™ Firming Treatment Mask',
 'Moisture Surge Overnight Mask',
 'Dermask Water Jet Vital Hydra Solution™',
 'Superfood Skin Reset Antioxidant Mask',
 'SUPERMASK - The Charcoal Mask',
 'After-Sun Mask - Watermelon',
 'BUBBLESHEET™ Oxygenating Deep Cleanse Mask',
 'Hand Mask',
 'Vitamin C Glow Face Mask',
 'Dermask Micro Jet Clearing Solution™',
 'SUPERMASK - The Bubble Mask',
 'Shake & Shot™ Rubber Hydro Mask',
 'Hello Calm™ Relaxing & Hydrating Face Mask with Hemp Seed Oil',
 'Moisturizing Mask',
 'Luminous Deep Hydration Lifting Mask',
 'Brightening Derm Revival Mask',
 'Mini Pore Waterclay Mask',
 'Intense Hydration Mask',
 'Hydrating Lip Mask',
 'Dermask Intra Jet Wrinkless Solution™',
 'Brighten Pineapple Enzyme + Gemstone Instant Glow Mask',
 'Benefiance Pure Retinol Intensive Revitalizing Face Mask',
 'Nose Strip',
 'Extra-Firming Mask',
 'Banana Bright Eye Crème',
 'Moisturizing Eye Bomb',
 'Eye Mask',
 'Advanced Night Repair Eye Supercharged Complex',
 'Benefiance WrinkleResist24 Pure Retinol Express Smoothing Eye Mask',
 'Baggage Claim Eye Masks',
 'The Pearl Tinted Eye Illuminating Treatment',
 'All About Eyes',
 'SEA Pack Your Bags Undereye Patches',
 'Resveratrol Lift Hyaluronic Acid Eye Lifting Balm',
 'Baggage Claim Rose Gold Eye Masks',
 'R.N.A. POWER Anti-Aging Eye Cream',
 'Eye Sleeping Mask',
 'Eye Mask - Cucumber - Anti-Puffiness',
 'Maracuja C-Brighter™ Eye Treatment',
 'Magic Eye Rescue Cream',
 'Eye Doctor™ Moisture Care For Skin Around Eyes',
 'VineActiv Vitamin C Energizing Eye Cream',
 'Luminous Deep Hydration Firming Eye Serum',
 'Superfood Hydrate + Firm Peptide Eye Cream',
 'Extra Eye Repair Cream',
 'Ferulic + Retinol Eye Cream',
 'Premier Cru Anti-Aging Eye Cream',
 'Hypoallergenic Firming Eye Cream',
 'Repairwear™ Anti-Gravity Eye Cream',
 'Essential Fx Acyl-Glutathione Eyelid Lift Serum',
 'GENIUS Ultimate Anti-Aging Eye Cream',
 'Future Solution LX Eye and Lip Contour Regenerating Cream',
 'Eye Contour Lifting Cream with Diamond Powder',
 'Water Bank Eye Gel',
 'All About Eyes Serum De-Puffing Eye Massage',
 'One Essential Eye Serum',
 'Pep-Start Eye Cream',
 'Essential Fx Acyl-Glutathione Smoothing & Brightening Under-Eye Cream',
 'White Lucent Anti-Dark Circles Eye Cream',
 'Bio-Performance LiftDynamic Eye Treatment',
 'MOISTURE BOUND Rejuvenating Eye Treatment Gel',
 'Renewing Eye Cream',
 'Peat Miracle Revital Eye Cream',
 'Vitamin C+ Collagen Brighten & Firm Eye Cream',
 'Eye Fuel',
 'Intensive Vitalizing Eye Essence',
 'POWER Advanced Wrinkle Fighter 360° Eye Serum',
 'Super Restorative Total Eye Concentrate',
 'Advanced Night Repair Eye Concentrate Matrix',
 'Cold Plasma+ Eye',
 'Retinol Firming Eye Cream',
 'Line Release Under Eye Repair Cream',
 'Essential-C Eye Cream SPF 15 PA++',
 'Multi-Task Eye Serum Mask',
 'Crème Ancienne® Eye Cream',
 'Wing Man Liver Detox and Dark Circle Supplement',
 'Hyaluronic Marine Dew It Right Eye Gel',
 'Youth Dose Eye Treatment',
 'Ultimate Revival Eye Cream',
 'Advanced Time Zone Age Reversing Line/Wrinkle Eye Creme',
 'SpectraLite EyeCare Pro LED Device',
 'Ultimate Miracle Worker Eye Cream',
 'Hydro-Dynamic® Ultimate Moisture For Eyes',
 'Hydra Life Cooling Hydration Sorbet Eye Gel',
 'High Potency Classics: Firming Eye Lift',
 'Gold Rush Eye Cream',
 'Enhancing Eye Lift Serum',
 'Noni Radiant Eye Oil',
 'Multi-Active Eye',
 'ELEVATE Firming & Lifting Contouring Eye Cream',
 'Essential Energy Eye Definer',
 'Eye Contour Gel',
 'DayWear Eye Cooling Anti-Oxidant Moisture GelCreme',
 'Neuropeptide Firming & Illuminating Under-Eye Cream',
 'Divine Eyes',
 'Vitamin C Ester Brightening Eye Serum',
 'Wrinkle Warrior® Eye Gel Visible Dark Circle Eraser',
 'Bio Performance Glow Revival Eye Treatment',
 'Reviving Eye Cream',
 'Crème Ancienne Supreme Eye Serum',
 'Repairwear Laser Focus Wrinkle Correcting Eye Cream',
 'Age Arrest Eye Cream',
 'High Performance Continuous Moisture Eye Cream',
 'Time Retreat Eye Treatment',
 'Urban Environment Oil-Free UV Protector Broad Spectrum Face Sunscreen SPF 42',
 'Tinted Moisturizer Broad Spectrum SPF 20 - Oil Free',
 'Silken Pore Perfecting Sunscreen Broad Spectrum SPF 35 PA+++',
 'Amazonian Clay BB Tinted Moisturizer Broad Spectrum SPF 20 Sunscreen',
 'Color Control Cushion Compact Broad Spectrum SPF 50+',
 'Double-Duty Face Moisturizer Broad Spectrum SPF 20',
 'Essential-C Day Moisture Broad Spectrum SPF 30 PA+++',
 'Moisture Surge CC Cream Hydrating Colour Corrector Broad Spectrum SPF 30',
 'Ultimate Miracle Worker Multi-Rejuvenating Cream SPF 30',
 'Powerful Wrinkle Reducing Cream Sunscreen Broad Spectrum SPF 30',
 'Repairwear Uplifting Firming Cream Broad Spectrum SPF 15',
 'Super Multi-Corrective Cream Sunscreen Broad Spectrum SPF 30',
 'Sunscreen Multi-Protection Broad Spectrum SPF 50',
 'DayWear Multi-Protection Anti-Oxidant Sheer Tint Release Moisturizer Broad Spectrum SPF 15 - Normal/ Combination Skin',
 'Dr. Andrew Weil For Origins™ Mega-Bright SPF 30 Skin Tone Correcting Oil-Free Moisturizer',
 'Smart Broad Spectrum SPF 15 Custom-Repair Moisturizer for Very Dry to Dry Skin',
 'Broad Spectrum SPF 50 Sunscreen Face Cream',
 'Oil-Free Sun Guard Sunscreen Water Resistant SPF 45',
 'Self Tanning Drops',
 'THE FACE Illuminating Self-Tan Drops',
 'THE BODY Illuminating Self-Tan Drops',
 'Self-Tanning Water',
 'Self Tan Purity Bronzing Water Face Mist',
 'Self Tan Classic Bronzing Mousse',
 'Tinted Self-Tanning Body Mist',
 'THE BUTTER Illuminating Tanning Butter',
 'Alpha Beta® Intense Glow Pad Self-Tanner for Face',
 'Lip Sleeping Mask',
 'Essential Lip Enhancer Balm',
 'Lip Balm & Scrub',
 'Brazilian Kiss Cupuaçu Lip Butter',
 'Intense Therapy Lip Balm SPF 25',
 'SEA Quench Lip Rescue Balm',
 'Camellia Gold Spun Lip Balm',
 'Le Rouge Perfecto Beautifying Lip Balm',
 'Refresh and Prep Lip Scrubber',
 'Future Solution LX Eye and Lip Contour Regenerating Cream',
 'Rosebud Salve',
 'Je Ne Sais Quoi Hydrating Lip Balm Treatment',
 'Chubby Stick Intense Moisturizing Lip Colour Balm',
 'Coconut Lip Glossy',
 'Hydrating Lip Balm',
 'Peach Lip Scrub - Peaches and Cream Collection',
 'Hydrating Lip Mask',
 'Lip Conditioner',
 'Sugar Sport Treatment Sunscreen SPF 30',
 'Hyaluronic Marine Collagen Lip Cushion',
 'Strawberry Lip Balm',
 'Pout Prep Lip Exfoliant',
 'Menthol And Eucalyptus Lip Balm',
 'KissKiss Liplift Lipstick Primer',
 'All About Lips',
 'Peach Mega Moisture Lip Balm - Peaches and Cream Collection',
 'Minted Rose Lip Balm',
 'KissKiss Roselip - Tinted Lip Balm',
 'Superbalm Lip Treatment',
 'Love Oil For Lips Glow-Infusing Lip Treatment',
 'Liplux SPF 30',
 'Tropical Ambrosia Lip Balm',
 'Neo Nude Ecstasy Balm',
 'Coconut Lip Scrubby',
 'Dual Nourishing Lip Serum',
 'Lip Sleeping Mask',
 'Niacinamide 10% + Zinc 1%',
 'Brazilian Bum Bum Cream',
 'Hyaluronic Acid 2% + B5',
 'The Water Cream Oil-Free Pore Minimizing Moisturizer',
 '"Buffet"',
 'Self Tanning Drops',
 'Natural Moisturizing Factors + HA',
 'Jet Lag Mask',
 'Caffeine Solution 5% + EGCG',
 'Azelaic Acid Suspension 10%',
 'Alpha Arbutin 2% + HA',
 'Hand & Nail Cream Trio',
 '100% Organic Cold-Pressed Rose Hip Seed Oil',
 'Lactic Acid 10% + HA',
 'Glycolic Acid 7% Toning Solution',
 'Mini Brazilian Bum Bum Cream',
 'Brazilian Crush Body Fragrance Mist',
 'Granactive Retinoid* 2% Emulsion',
 'C-Rush™ Vitamin C Gel Moisturizer',
 'The Rice Polish Foaming Enzyme Powder',
 'Water Sleeping Mask',
 'Brazilian 4 Play Moisturizing Shower Cream-Gel',
 'EDP Cheirosa ’62 Holiday Fragrance Set',
 'The Essence Plumping Skin Softener',
 'Goodnight Glow Retin-ALT Sleeping Crème',
 'Jet Lag Mask Mini',
 'Blue Tansy BHA and Enzyme Pore Refining Mask',
 'Balancing Force™ Oil Control Toner',
 'Beauty Elixir',
 'Transforming Walnut Scrub',
 'Pure One Step Camellia Oil Cleanser',
 'Whipped Argan Oil Body Butter',
 'Vitamin C Suspension 23% + HA Spheres 2%',
 'Vitamin C Suspension 30% in Silicone',
 'Clear Improvement® Active Charcoal Mask to Clear Pores',
 'Let It Grow - Hair Sweet Hair Vegan Gummies Duo Set',
 'Grape Water',
 'Find Your Balance™ Oil Control Cleanser',
 'Alpha Beta® Ultra Gentle Daily Peel for Sensitive Skin',
 'Brazilian Crush Body Fragrance Mist Mini',
 'Noni Glow Face Oil',
 'Pore-Balance™ Facial Sauna Scrub',
 'Counter Balance™ Oil Control Hydrator',
 'High-Potency Night-a-Mins™ Resurfacing Cream with Fruit-Derived AHAs',
 'Vinopure Natural Salicylic Acid Pore Minimizing Toner',
 'Glow Cycle Retin-ALT Power Serum',
 'Self-Tanning Water',
 'Overtime Mask',
 'Instant Detox Mask',
 'Coco Rose Exfoliating Body Scrub']

In [ ]:
def corrected_urls(names_with_incorrect_urls):
    chrome_path = '/Users/marissakhoury/Desktop/webscrape/drivers/chromedriver'
    # initiate empty dataframe
    df = pd.DataFrame(columns=['prod_name', 'URL'])    
    for name in names_with_incorrect_urls:
        driver = webdriver.Chrome(executable_path = chrome_path)
        driver.get('https://www.sephora.com')
        time.sleep(3)
        driver.find_element_by_xpath('/html/body/div[1]/div[1]/header/div[1]/div[1]/div/div[1]/div/form/div/div/div/div/input').send_keys(name, Keys.ENTER)
        time.sleep(2)
        try:
            x_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div/div/button')
            ActionChains(driver).click(x_button).perform()
            time.sleep(2)
        except NoSuchElementException:
            pass
        time.sleep(1)
        element_results = driver.find_elements_by_class_name('css-ix8km1')
        if not element_results:
            # assume it went directly to the page
            corrected_url = driver.current_url
        else:
            corrected_url = element_results[0].get_attribute('href') # url for this product, assume it's the first result

    # transform into a data frame
        driver.close()
        dic = {'prod_name': name, 'URL': corrected_url}
        print(dic)
        df = df.append(dic, ignore_index = True)
    print(df)
    # add columns
    df2 = pd.DataFrame(columns=['brand', 'name', 'price', 'rank', 'skin_type', 'ingredients'])
    df = pd.concat([df, df2], axis = 1)
    
    return df
corrected_df = corrected_urls(names_with_incorrect_urls)

In [8]:
corrected_df.to_csv('corrected_df.csv')

In [ ]:
corrected_df = pd.DataFrame.from_csv('corrected_df.csv')